In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.5.0
2.5.0


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 20

# Wikilabels
df = pd.read_csv('../data/wiki/wiki_name_race.csv')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df

# Additional features
sdf['name_first'] = sdf.name_first.str.title()
sdf['name_last'] = sdf.name_last.str.title()

rdf = sdf.groupby('race').agg({'name_first': 'count'})
rdf

,name_first
race,
"Asian,GreaterEastAsian,EastAsian",5497
"Asian,GreaterEastAsian,Japanese",7333
"Asian,IndianSubContinent",7861
"GreaterAfrican,Africans",3672
"GreaterAfrican,Muslim",6242
"GreaterEuropean,British",41445
"GreaterEuropean,EastEuropean",8329
"GreaterEuropean,Jewish",10239
"GreaterEuropean,WestEuropean,French",12293


In [3]:
rdf.to_csv('./wiki/lstm/wiki_name_race.csv', columns=[])

## Preprocessing the input data

In [4]:
# concat last name and first name
sdf['name_last_name_first'] = sdf['name_last'] + ' ' + sdf['name_first']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 2210
Max feature len = 74, Avg. feature len = 12


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [5]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 25 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

107097 train sequences
26775 test sequences
Pad sequences (samples x time)
X_train shape: (107097, 25)
X_test shape: (26775, 25)
13 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (107097, 13)
y_test shape: (26775, 13)


In [6]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 32)            70720     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 13)                1677      
Total params: 154,829
Trainable params: 154,829
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/20
3013/3013 [==============================] - 141s 38ms/step - loss: 1.6700 - accuracy: 0.4921 - val_loss: 1.1365 - val_accuracy: 0.6692
Epoch 2/20
3013/3013 [==============================] - 113s 38ms/step - loss: 1.1289 - accuracy: 0.6689 - val_loss: 1.0618 - val_accuracy: 0.6923
Epoch 3/20
3013/3013 [==============================] - 113s 38ms/step - loss: 1.0534 - accuracy: 0.6913 - val_loss: 1.0353 - val_accuracy: 0.6999
Epoch 4/20
3013/3013 [==============================] - 113s 38ms/step - loss: 1.0160 - accuracy: 0.7051 - val_loss: 1.0087 - val_accuracy: 0.7110
Epoch 5/20
3013/3013 [==============================] - 113s 38ms/step - loss: 0.9787 - accuracy: 0.7167 - val_loss: 0.9938 - val_accuracy: 0.7157
Epoch 6/20
3013/3013 [==============================] - 113s 38ms/step - loss: 0.9484 - accuracy: 0.7250 - val_loss: 0.9810 - val_accuracy: 0.7203
Epoch 7/20
3013/3013 [==============================] - 113s 38ms/step - loss: 0.9272 - accuracy: 0.7312 - va

## Confusion Matrix

In [8]:
p = model.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

837/837 - 6s
                                       precision    recall  f1-score   support

     Asian,GreaterEastAsian,EastAsian       0.87      0.80      0.83      1099
      Asian,GreaterEastAsian,Japanese       0.89      0.90      0.89      1467
             Asian,IndianSubContinent       0.82      0.75      0.78      1572
              GreaterAfrican,Africans       0.59      0.43      0.50       734
                GreaterAfrican,Muslim       0.66      0.72      0.69      1248
              GreaterEuropean,British       0.76      0.89      0.82      8289
         GreaterEuropean,EastEuropean       0.78      0.74      0.76      1666
               GreaterEuropean,Jewish       0.50      0.46      0.48      2048
  GreaterEuropean,WestEuropean,French       0.72      0.59      0.65      2459
GreaterEuropean,WestEuropean,Germanic       0.52      0.43      0.47       774
GreaterEuropean,WestEuropean,Hispanic       0.73      0.69      0.71      2082
 GreaterEuropean,WestEuropean,Italian 

## Save model

In [9]:
model.save('./wiki/lstm/wiki_name_lstm.h5')

In [10]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./wiki/lstm/wiki_name_vocab.csv', index=False, encoding='utf-8')